# Learning to use a vending machine

Sometimes users must perform actions in a certain order if they want reward.

We will be base this tutorial on an experiment conduct by Daw et al., 2005, with rats... which are well known users of vending machines. We will refer to rats/users inter-changeably for this tutorial.

Here is an image from the paper showing the state transitions of the task. The user starts in state S0.

<img src="states.png" width="400" height="400">

As you can see, pressing the lever, followed by putting your hand in the magazine provides a reward (a chocolate bar). Doing the actions in any other order, or missing an action, fails to lead to reward.

This tutorial will teach you how to create a Reinfocement Learning model of this task in Python. We will use Q-learning to determine the best move to make.

### Jupyter Notebook Summary

This tutorial has been produced using Jupyter Notebook. You will sections of text, accompanied by sections of code. To run the code, click on it, hold shift, and press enter.

## Building the Model

#### Setting up the experiment

We will first write  code to set up the reward array and Q-value array. The code used in this tutorial has been based on Mic (2016), though it has been heavily adapted and rewritten in order to suit this tutorial.

Each section in the tutorial will begin with a block of code, followed by a line by line explanation of the code. The code most likely won't make much sense before reading the explanations.

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    def __init__(self):
        
        # keep track of current state
        self.current_state = -1

        # array for storing reward values
        self.r = np.array([[-1,  -10,  -30,   -1],   # State S0
                           [-1,   -1,  -30,   90],   # State S1
                           [-1,   -1,   -1,   -1],   # State S2
                           [-1,   -1,   -1,   -1]])  # State S3

        # array for storing q values
        self.q = np.zeros_like(self.r)
        
        # keep track of total reward received
        self.total_distal_reward = 0.0
        
        # keep track of if we in the proximal state
        self.proximal = 0

        # keep track of rewards after proximal state
        self.total_proximal_reward = 0.0
        
        print "Reward array: \n", self.r
        print "Q-value array: \n", self.q

experiment_instance = Machine()        

Reward array: 
[[ -1 -10 -30  -1]
 [ -1  -1 -30  90]
 [ -1  -1  -1  -1]
 [ -1  -1  -1  -1]]
Q-value array: 
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


Next, we create a class named 'Machine' to hold our functions.

An initial '\__init\__' function has been created. This is used to set up the experiment.

    self.current_state = -1

This variable is used to keep track of the current state.

    self.r = np.array([[-1,  -10,  -30,   -1],   # State S0
                       [-1,   -1,  -30,   90],   # State S1
                       [-1,   -1,   -1,   -1],   # State S2
                       [-1,   -1,   -1,   -1]])  # State S3

This line sets up the reward matrix that defines a reward for each state transition.

When in a given state, the user has a choice of actions. Choosing an action gives a reward. For example, looking at the graph above, if the user is in state S1 and chooses the action 'Enter magazine' (S1 -> S3), a positive reward will be given (food is obtained).

The reward matrix corresponds to the actions and rewards received when starting in a certain state. This is better explained through an example:

We put -1 where moves are not possible. Therefore positions 0 and 3 in  S0 have a value of -1.

If we move into S2, we give a negative reward of -30 as this is a terminal state, and the user receives nothing for this run of the experiment. Therefore position 2 in array 0, moving from S0 to S2, has a value of -30.

If we move from S0 to S1, we will assign a negative cost of -10. This negative cost can be seen as the time taken, or the energy needed to make this transition.

For state S1, moving to the terminal state of S2 has a reward of -30, as before. Moving to S3 means that the user receives a reward of food, and therefore we give it a large positive reward of 90.

We can't move anywhere from S2 or S3, so we assign -1 to every transition from S2 and S3.

It can be noted that these rewards vary from the rewards in the above graph, which consist only of 0s and a 1. Arguably, the rewards in our code better reflect the nature of the problem - there should be a negative reward for bad actions. In a logical format, the user spending a lot of time before it is able to eat is, in a sense, a bad thing and therefore deserves a negative reward.

    self.q = np.zeros_like(self.r)
    
This creates an array of the same dimensions as r and fills it with zeros.
    
To understand the use of this, Q-learning needs to be explained a little.

Q-learning keeps track the reward gained for picking each action in each state, and updates a Q-value which helps the user to decide which actions are the best. 
Q-values come in state-action pairs, and tell us how good picking that particular action while in that particular state is. The higher the Q-value, the better the user thinks picking that action in that state is.
The update algorithm will be explained later. The Q-table stores the Q-values.

As the user has no knowledge of the states, actions, and rewards at the start, all the Q-values in the Q-table are initialised to zero.

    self.total_distal_reward = 0.0

We use to keep track of the total reward received from taking actions starting from the distal state, the state furthest from the reward, S0. This is equivalent to the total reward received throughout the experiment. We use this to analyse results later.

    self.proximal = 0

We use this to keep track of if we are in the proximal state - the state one step from the reward. In this experiment, this state is S1. We will set this to 1 if we are in S1.

    self.total_proximal_reward = 0.0

We use this to keep track of the total reward accumulated from taking an action in the proximal state, S1. We use this to analyse results later.

Running the code will print out the reward table and Q-table.

#### Getting the available actions

We will now write a function for getting available actions - the actions that are possible when starting in a certain state. In this tutorial, an action is the state we choose to move to. For example, if we are starting in S1, the available actions are 2 and 3, where choosing action 2 will move to S2, providing a reward of -30, and action 3 will move to S3, providing a reward of 100.

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    
    from code import __init__  # import init written previously

    def get_available_actions(self):

        # get the row of rewards for the current state
        current_reward_row = self.r[self.current_state]
        
        print "current_reward_row: ", current_reward_row

        # get the states which are possible to transition to
        current_available_actions = np.where(current_reward_row != -1)[0]
        
        print "\ncurrent_available_actions: ", current_available_actions

        return current_available_actions
    
experiment_instance = Machine()
experiment_instance.current_state = 0
experiment_instance.get_available_actions()

current_reward_row:  [ -1 -10 -30  -1]

current_available_actions:  [1 2]


array([1, 2], dtype=int64)

    from code import __init__

As in the first tutorial, we've added this to our code. We have a separate file, 'code.py', that contains all the functions we will write. We wrote '\__init\__' in the previous section. To save space, we can replace the entire '\__init\__' with 'from code import \__init\__', which takes '\__init\__' from code.py and puts it where the import line is.

We will be adding more functions to this line throughout the tutorial. Functions can be added by separating them with a comma.

As you can see, we've now added the 'get_available_actions' function to our code. Let's walk through it.

    def get_available_actions(self):

The function takes in the experiment instance as a parameter (something you must give the function for it to run).

    current_reward_row = self.r[self.current_state]
    
This line of code gets the array of rewards for taking actions in current_state. 

    array_name[n]
    
will get the the nth element from array_name.

Therefore, if current_state is 0, it will get element zero from the reward array. Elements are the things that the array contains. Remember, the reward array is an array of arrays, so the element is also array.

For example, if the current state is S0, this will get [-1,    -10,  -30,   -1].

Running the code will display current_reward_row if the current state is 0. You can set current_state to another state to see the results for different states.

    current_available_actions = np.where(current_reward_row != -1)[0]

This returns the available actions.

    np.where(condition)
    
returns the indexes of where the condition is true.

    current_reward_row != -1
    
is our condition. Remember that when a move is not possible, we have -1 as the reward. != means is not equal.

    np.where(current_reward_row != -1)

Therefore, this line of code returns all the indexes where the reward is not equal to -1.

The result is returned in a tuple: (array0, array1). Only when the array is 2D is array1 used. Since our array is 1D, array1 is empty, and we don't need to worry about it, so we can just use [0] to get the first element in the tuple (the available actions). This part isn't particularly important, just remember to use the [0].

Run the code to see how this part of the code works.

    return current_available_actions

This returns the current available actions.

#### Choosing the next action

This section uses  'epsilon-greedy' to choose which action to take next. We covered the algorithm in the previous tutorial.

As a quick reminder, the user needs to choose to explore or exploit. The epsilon variable adjusts how much the user chooses to explore vs exploit. For a full explanation, see the first tutorial.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    
    from code import __init__, get_available_actions

    def choose_next_action(self, available_actions, epsilon):

        # first calculate best action

        # used to hold the q values of possible moves
        possible_q_list = np.zeros(len(self.q[self.current_state]))

        # set each of the elements to 'None'
        for x in range(len(possible_q_list)):
            possible_q_list[x] = None

        # where a move is possible, add the Q-value to possible_q_list
        possible_q_list[available_actions] = self.q[self.current_state, available_actions]

        # possible_q_list contains the Q-values for moves from that state which are possible

        # the moves with the largest Q-values, ignoring 'None'
        best_action_list = np.where(possible_q_list == np.nanmax(possible_q_list))[0]

        print "Best action list ", best_action_list

        # choose a best action randomly in case there are multiple
        best_action = random.choice(best_action_list)

        # generate random float between 0 and 1
        rand_num = random.random()

        # if greater than epsilon, return best action
        if rand_num > epsilon:

            print "Exploit: ", best_action
            return best_action

        # else, return a random other action
        else:
            random_action = random.choice(available_actions)

            while random_action == best_action:
                random_action = random.choice(available_actions)

            print "Explore: ", random_action
            return random_action
        
experiment_instance = Machine()
experiment_instance.current_state = 0
available_actions = experiment_instance.get_available_actions()
print "Available actions: ", available_actions
print experiment_instance.choose_next_action(available_actions, 0.2)

Let's break this down.

    def choose_next_action(self, available_actions, epsilon):

The function takes in the available actions calculated in the previous function, and epsilon as parameters, so it can choose the amount it explores vs exploits. Epsilon is a float (decimal number) between 0.0 and 1.0. 

    possible_q_list = np.zeros(len(self.q[self.current_state]))
    
This creates an array of zeros with the same number of entries as the number of Q-values for the current state. In our example this will create an array [0, 0, 0, 0] each time.

    for x in range(len(possible_q_list)):
        possible_q_list[x] = None
        
Loop through the possible_q_list and assign 'None' to each of its elements. This ends up with possible_q_list being [None, None, None, None].

    possible_q_list[available_actions] = self.q[current_state, available_actions]
    
We have previously calculated an array of available_actions - the possible actions that can be taken. For example, from S0, the array will be [1, 2]. We get the value from the Q-table for each of these actions, and place it the corresponding index in possible_q_list. For example, if the Q-table at the current_state S0 is [0, 5, 10, 0], possible_q_list will end up as [None, 5, 10, 0].

    best_action_list = np.where(possible_q_list == np.nanmax(possible_q_list))[0]
    
Let's take this step by step.

    np.nanmax(possible_q_list)

returns the largest number in 'possible_q_list', ignoring where there is 'None'. For example, if possible_q_list is [0, None, 5, 5], 5 would be returned.

    np.where(possible_q_list == np.nanmax(possible_q_list))
    
returns the indexes of possible_q_list where the Q-value at that index is the equal to the number returned by np.nanmax(possible_q_list). 

This array of numbers is assigned to 'best_action_list'. There can be a single number in this array, or multiple, if there are multiple possible moves with the same highest Q-value.

The [0] is placed at the end as explained in the previous function.

For example, if possible_q_list is [0, None, 5, 5], then np.nanmax(possible_q_list) would return 5, and np.where(possible_q_list == np.nanmax(possible_q_list))[0] would return [2, 3], as the elements at index 2 and 3 are 5.

    best_action = random.choice(best_action_list)

best_action is a randomly picked from best_action_list. This is in case there are multiple 'best actions'. In the example given above, this would randomly return either 2 or 3. If there is only one entry in best_action_list, that entry will be chosen.

    rand_num = random.random()

This generates a random float between 0.0 and 1.0, for use in epsilon-greedy.

    if rand_num > epsilon:

If the random number is greater than the epsilon value, we exploit. We calculate what the user thinks is the current best action for the state that it is in, the action with the highest Q-value currently. As explained earlier, Q-values come in state-action pairs, and tell us how good picking that particular action while in that particular state is. The higher the Q-value, the better the user thinks picking that action in that state is.

    return best_action

Return the action.

    else:
        random_action = random.choice(available_actions)

        while random_action == best_action:
            random_action = random.choice(available_actions)

        return random_action
        
If the random number is not greater than epsilon, we explore, returning a random possible action.

We do this by randomly selecting an action. If this action is the best action, we enter the 'while' loop. We keep performing the code in the 'while' loop, choosing a new random action, until the chosen action is not the best action.

We then return it.

This ensures that the action returned when we explore cannot be the best action.

Note that there are only two choices in action for each of the states with a choice of action. We could just return the other action, but the code above works with problems with states with more actions.

#### Updating the Q-values

We need to update the Q-values after each action. We have added two functions, calculate_max_q_value, and update_q. 

To understand this part, you need to understand how Q-values are updated in Q-learning.

This is the formula for updating:

$$Q(s_{t}, a_{t}) \gets Q(s_{t}, a_{t}) + \alpha \cdot (r_{t+1} + \gamma \cdot maxQ(s_{t+1}, a) - Q(s_{t}, a_{t}))$$

The above is taken from Sutton and Barto, 1998, pp.148, which is a good resource on reinforcement learning.

In words, this can be written:

$$Q_{current} \gets Q_{current} + \alpha \cdot (reward\_received + \gamma \cdot maxQ_{next\_state} - Q_{current})$$



$\alpha$ , or alpha, is known as the learning rate. This is how much weight the model puts on new information - how much the updated Q-value is allowed to change from the previous Q-value. This is usually set to a high value at the start, and slowly decays over time, though in this tutorial we will just keep it at a flat value for simplicity. 

- As an exercise, you could implement a decaying alpha.

$\gamma$, or gamma, is known as the discount factor. This is how much importance the user places on immediate rewards vs future rewards. A discount factor of 0 means the user will only consider immediate rewards, while a higher gamma gives greater importance to future rewards. This is less important in our example, since we have few states and rewards, but is far more important in larger examples with more states and rewards.

$maxQ_{next\_state}$ is largest Q-value available in the next state. For example, if we are in S0 and pick the action to move  to S1, this is the maximum Q-value in S1.


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    
    from code import __init__, get_available_actions, choose_next_action

    def calculate_max_q_value(self, action):

        # get the max q value of the next state
        max_value = np.amax(self.q[action])

        return max_value

    def update_q(self, action, alpha, gamma):

        # current q value
        current_q = self.q[self.current_state, action]
        
        print "Current Q-value: ", current_q

        # current reward value
        current_r = self.r[self.current_state, action]

        print "Reward received: ", current_r

        # total reward received
        self.total_distal_reward += current_r

        if self.proximal == 1:

            self.total_proximal_reward += current_r

        # max q value at next state
        max_q_value = self.calculate_max_q_value(action)

        print "Max Q value in next state: ", max_q_value

        # update current q value
        self.q[self.current_state][action] = current_q + alpha * (current_r + gamma * max_q_value - current_q)

        print "Updated Q value table: "

        print self.q
        
experiment_instance = Machine()
experiment_instance.current_state = 0
experiment_instance.update_q(2, 0.1, 0.8)

Let's see how this works.

    def calculate_max_q_value(self, action):
    
This function calculates the maximum Q-value in the next state. It takes in the action chosen as a parameter. The action chosen corresponds to the next state.

    max_value = np.amax(self.q[action])

    return max_value
    
    
self.q[action] is the array of Q-values for the next state (the next state is the same as the action chosen).
        
The function simply gets the maximum Q-value from the next state, using the amax function, and returns it. For example, if the Q-value array for the next state is [0, 2, 5, 5], 5 will be returned.

    def update_q(self, action, alpha, gamma):
    
This action updates the Q-value, using the algorithm spoken about above. It takes in the action chosen, alpha, and gamma as parameters.

    current_q = self.q[self.current_state, action]

Firstly, we need the current Q-value. This is uses current_state to get the corresponding Q-value array for the current state from the Q-table, and then uses 'action' to get the corresponding value from that array.

For example, if the Q-value array for S0 is [0, 10, 5, 0] and we have chosen to move to S1, 10 will be selected.

    current_r = self.r[self.current_state, action]
    
This does the same thing as above with the reward array to get the reward for choosing this particular action in this particular state.

    self.total_distal_reward += current_r
    
This adds the current reward to the total_distal_reward variable. Remember that this will keep track of the total reward gained during the experiment.

    if self.proximal == 1:

            self.total_proximal_reward += current_r
            
          
If we are in the proximal state, we add the current reward to the total_proximal_reward. Remember that total_proximal_reward is the total reward achieved when starting in a proximal state.

    max_q_value = self.calculate_max_q_value(action)
 
This calls the calculate_max_q_value function to get the maximum Q-value in the next state.

    self.q[self.current_state][action] = current_q + alpha * (current_r + gamma * max_q_value - current_q)
    
This is the algorithm explained above. This updates the Q-value. Make sure this makes sense to you.

Running the code will update the Q-matrix from all zeros for if we are starting in S0 and choose to move to S2.

#### Running an experiment

We now have all the code we need to run experiments, and we can finish off the class.

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    
    from code import __init__, get_available_actions, choose_next_action, calculate_max_q_value, update_q
    
    def run_experiment(self, initial_state, alpha, gamma, epsilon, num_rewarded_runs_wanted):

        # counter for number of runs
        num_rewarded_runs = 0
        num_runs = 0
        num_proximal_runs = 0
        self.current_state = initial_state

        total_distal_reward_array = np.array([])
        total_proximal_reward_array = np.array([])

        while num_rewarded_runs < num_rewarded_runs_wanted:

            print ""
            print "Run number: ", num_runs

            # if a terminal state is reached, reset to intial state, and increment counter
            if self.current_state == 2 or self.current_state == 3:

                num_runs += 1

                if self.current_state == 3:
                    
                    num_rewarded_runs += 1

                    total_distal_reward_array = np.append(total_distal_reward_array,
                                                          self.total_distal_reward / (80 * num_runs))

                    total_proximal_reward_array = np.append(total_proximal_reward_array,
                                                          self.total_proximal_reward / (90 * num_proximal_runs))

                self.current_state = initial_state

                self.proximal = 0

            print "Current state: ", self.current_state

            # get available action
            available_actions = self.get_available_actions()
            print "Available actions: ", available_actions

            # pick action from available actions
            action_chosen = self.choose_next_action(available_actions, epsilon)
            print "Action chosen ", action_chosen

            # update q matrix
            self.update_q(action_chosen, alpha, gamma)

            # current state becomes the new chosen state
            self.current_state = action_chosen

            if self.current_state == 1:

                num_proximal_runs += 1

                self.proximal = 1

        print "\nFinal Q Matrix"
        print self.q

        plt.figure()
        plt.plot(total_distal_reward_array, label="Expected Reward (Distal)", color='orange')  # distal == total
        plt.draw()
        plt.ylim(0, 1)  # limits for the y-axis
        plt.legend(loc='best')  # display the graph key
        plt.savefig('distal_graph.png')  # save graph to folder

        plt.figure()
        plt.plot(total_proximal_reward_array, label="Expected Reward (Proximal)", color = 'orange')
        plt.draw()
        plt.ylim(0, 1)  # limits for the y-axis
        plt.legend(loc='best')  # display the graph key
        plt.savefig('proximal_graph.png')  # save graph to folder

        plt.show()  # display the graph

Let's break down the final part of the class. This section is not runnable, as we will go through how to run it in the next section.

    def run_experiment(self, initial_state, alpha, gamma, epsilon, num_rewarded_runs_wanted):
    
The parameters are the initial state (the state you want the user to start in), alpha (learning factor), gamma (discount rate), and epsilon, and the number of rewarded runs wanted.

A rewarded run is when the user reaches S3.

    num_rewarded_runs = 0
    num_runs = 0
    num_proximal_runs = 0
    
These are variables to keep track of the total number of rewarded runs, the total number of runs (times the uer has started from S0 and reaches S2 or S3), and the number of times the user has reached the proximal state.

    self.current_state = initial_state
    
We set the initial current state to the initial state specified.

    total_distal_reward_array = np.array([])
    total_proximal_reward_array = np.array([])

We use these to keep track of rewards from the distal and proximal state

    while num_rewarded_runs < num_rewarded_runs_wanted:
    
We keep looping through the experiment until we get the required number of rewarded runs.

    if self.current_state == 2 or self.current_state == 3:

        num_runs += 1

        if self.current_state == 3:

            num_rewarded_runs += 1

            total_distal_reward_array = np.append(total_distal_reward_array,
                                                  self.total_distal_reward / (80 * num_runs))

            total_proximal_reward_array = np.append(total_proximal_reward_array,
                                                  self.total_proximal_reward / (90 * num_proximal_runs))

    self.current_state = initial_state

    self.proximal = 0

If the user reaches S2 or S3, a run has been completed, and we increase num_runs by 1.

If the user reaches S3, we complete a rewarded run. We increase num_rewarded_runs by 1.

We also append to our arrays.

    np.append(array0, array1)

will append array1 to array0 (add array1 to the end of array0).

We append the total_distal_reward/(80 * num_runs) to the total_distal_reward_array. This is the average reward achieved per run throughout the course of the experiment, the expected value.

80 is the maximum reward from a single run of the experiment. A reward of 90 is received for moving from S1 to S3. Moving from S0 to S1 achieves a reward of -10. Therefore, maximum reward of a run is 80. The maximum reward during the course of the whole experiment is 80 * num_runs.

We do something similar for the total_proximal_reward_array. As the user starts in S1 here, the maximum reward is 90. The maximum reward for the total number of times it has entered the proximal state is 90 * num_proximal_runs.

    available_actions = self.get_available_actions(current_state)

This calls the get_available_actions function that we wrote earlier to get an array of possible actions from our current state.

    action_chosen = self.choose_next_action(available_actions, current_state, epsilon)
    
This uses the choose_next_action function we wrote to choose the next action using the epsilon-greedy algorithm.

    self.update_q(current_state, action_chosen, alpha, gamma)

We update the Q-value for current state-action chosen pair by calling our function.

    current_state = action_chosen
    
We move the user to the next state by setting the current state to the chosen action.

    if self.current_state == 1:

        num_proximal_runs += 1
 
        self.proximal = 1

If we are now in the proximal state, S1, we increase num_proximal_runs by 1, and set set proximal to 1.

    plt.figure()
    plt.plot(total_distal_reward_array, label="Expected Reward (Distal)", color='orange')
    plt.draw()
    plt.ylim(-0.5, 1)  # limits for the y-axis
    plt.legend(loc='best')  # display the graph key
    plt.savefig('distal_graph.png')  # save graph to folder
    
    plt.figure()
    plt.plot(total_proximal_reward_array, label="Expected Reward (Proximal)", color = 'orange')
    plt.draw()
    plt.ylim(-0.5, 1)  # limits for the y-axis
    plt.legend(loc='best')  # display the graph key
    plt.savefig('proximal_graph.png')  # save graph to folder

    plt.show()  # display the graph
    
This plots graphs for the expected values, using the matplotlib library, in the same way as the first tutorial.

We plot a graph for the distal and proximal expected rewards.

That's it, we've finished the class! Let's get some results.

#### Getting Results

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


class Machine:
    def __init__(self):

        # keep track of current state
        self.current_state = -1

        # array for storing reward values
        self.r = np.array([[-1,  -10,  -30,   -1],   # State S0
                           [-1,   -1,  -30,   90],   # State S1
                           [-1,   -1,   -1,   -1],   # State S2
                           [-1,   -1,   -1,   -1]])  # State S3

        # array for storing q values
        self.q = np.zeros_like(self.r)

        # keep track of total reward received
        self.total_distal_reward = 0.0

        # keep track of if we in the proximal state
        self.proximal = 0

        # keep track of rewards after proximal state
        self.total_proximal_reward = 0.0

    def get_available_actions(self):

        # get the row of rewards for the current state
        current_reward_row = self.r[self.current_state]

        # get the states which are possible to transition to
        current_available_actions = np.where(current_reward_row != -1)[0]

        return current_available_actions

    def choose_next_action(self, available_actions, epsilon):

        # first calculate best action

        # used to hold the q values of possible moves
        possible_q_list = np.zeros(len(self.q[self.current_state]))

        # set each of the elements to 'None'
        for x in range(len(possible_q_list)):
            possible_q_list[x] = None

        # where a move is possible, add the Q-value to possible_q_list
        possible_q_list[available_actions] = self.q[self.current_state, available_actions]

        # possible_q_list contains the Q-values for moves from that state which are possible

        # the moves with the largest Q-values, ignoring 'None'
        best_action_list = np.where(possible_q_list == np.nanmax(possible_q_list))[0]

        print "Best action list ", best_action_list

        # choose a best action randomly in case there are multiple
        best_action = random.choice(best_action_list)

        # generate random float between 0 and 1
        rand_num = random.random()

        # if greater than epsilon, return best action
        if rand_num > epsilon:

            print "Exploit: ", best_action
            return best_action

        # else, return a random other action
        else:
            random_action = random.choice(available_actions)

            while random_action == best_action:
                random_action = random.choice(available_actions)

            print "Explore: ", best_action
            return random_action

    def calculate_max_q_value(self, action):

        # get the max q value of the next state
        max_value = np.amax(self.q[action])

        return max_value

    def update_q(self, action, alpha, gamma):

        # current q value
        current_q = self.q[self.current_state, action]

        print "Current Q-value: ", current_q

        # current reward value
        current_r = self.r[self.current_state, action]

        print "Reward received: ", current_r

        # total reward received
        self.total_distal_reward += current_r

        if self.proximal == 1:

            self.total_proximal_reward += current_r

        # max q value at next state
        max_q_value = self.calculate_max_q_value(action)

        print "Max Q value in next state: ", max_q_value

        # update current q value
        self.q[self.current_state][action] = current_q + alpha * (current_r + gamma * max_q_value - current_q)

        print "Updated Q value table: "

        print self.q

    def run_experiment(self, initial_state, alpha, gamma, epsilon, num_rewarded_runs_wanted):

        # counter for number of runs
        num_rewarded_runs = 0
        num_runs = 0
        num_proximal_runs = 0
        self.current_state = initial_state

        total_distal_reward_array = np.array([])
        total_proximal_reward_array = np.array([])

        while num_rewarded_runs < num_rewarded_runs_wanted:

            print ""
            print "Run number: ", num_runs

            # if a terminal state is reached, reset to intial state, and increment counter
            if self.current_state == 2 or self.current_state == 3:

                num_runs += 1

                if self.current_state == 3:

                    num_rewarded_runs += 1

                    total_distal_reward_array = np.append(total_distal_reward_array,
                                                          self.total_distal_reward / (80 * num_runs))

                    total_proximal_reward_array = np.append(total_proximal_reward_array,
                                                          self.total_proximal_reward / (90 * num_proximal_runs))

                self.current_state = initial_state

                self.proximal = 0

            print "Current state: ", self.current_state

            # get available action
            available_actions = self.get_available_actions()
            print "Available actions: ", available_actions

            # pick action from available actions
            action_chosen = self.choose_next_action(available_actions, epsilon)
            print "Action chosen ", action_chosen

            # update q matrix
            self.update_q(action_chosen, alpha, gamma)

            # current state becomes the new chosen state
            self.current_state = action_chosen

            if self.current_state == 1:

                num_proximal_runs += 1

                self.proximal = 1

        print "\nFinal Q Matrix"
        print self.q

        plt.figure()
        plt.plot(total_distal_reward_array, label="Expected Reward (Distal)", color='orange')  # distal == total
        plt.draw()
        plt.ylim(-0.5, 1)  # limits for the y-axis
        plt.legend(loc='best')  # display the graph key
        plt.savefig('distal_graph.png')  # save graph to folder

        plt.figure()
        plt.plot(total_proximal_reward_array, label="Expected Reward (Proximal)", color = 'orange')
        plt.draw()
        plt.ylim(-0.5, 1)  # limits for the y-axis
        plt.legend(loc='best')  # display the graph key
        plt.savefig('proximal_graph.png')  # save graph to folder

        plt.show()  # display the graph

    def check_training(self):
        self.current_state = 0
        steps = [self.current_state]

        while self.current_state != 2 and self.current_state != 3:

            best_action_list = np.where(self.q[self.current_state] == np.amax(self.q[self.current_state]))[0]

            best_action = random.choice(best_action_list)

            steps.append(best_action)
            self.current_state = best_action

        # Print selected sequence of steps
        print("Selected path:")
        print(steps)

experiment_instance = Machine()
experiment_instance.run_experiment(0, 0.1, 0.8, 0.1, 120)

Here is the completed code. We've added one final bit.

    experiment_instance = Machine()
    Machine.run_experiment(experiment_instance, 0, 0.1, 0.8, 0.3, 120)

This creates an instance of the experiment and runs it with the given parameters:
The initial state is 0, alpha is 0.1, gamma is 0.8, epsilon is 0.3, and the number of runs is 120. We have used 120 runs in order to plot graphs with a similar axis to Daw et al., 2005.

There are a number of print statements throughout the code. This is so that when the code is run, it is easy to see exactly what is happening.

Run the code to get results. You may need to scroll through. The two graphs plotted will be saved in the same folder as this notebook as 'distal_graph.png' and 'proximal_graph.png'.

### Results

The final Q-values show that the model believes that choosing to go from S0 to S1, and S1 to S3 are good options, and comparing this to the diagram, we can see that this is correct. 
 
By scrolling through the results above, you can see how the Q-values change over time.

These are the results from the paper:

<img src="results.png" width="400" height="400">


S0 to S1, and S1 to S3 become far more likely after 50 rewards, much like our model. You can see this in the expected value of our graphs, or you can scroll through above and see how the Q-values in the table change over time.

Here is graph for the expected value from the distal state (S0), from the paper. Ignore the blue line.

<img src="distal_daw.png" width="400" height="400">

Compare your results to these.

Here is graph for the expected value from the proximal state (S0), from the paper. Ignore the blue line.

<img src="proximal_daw.png" width="400" height="400">

Compare your results to these.

As you can see, both graphs closely match the graphs provided by the paper.

The expected reward quickly increases in both graphs, but plateaus, due to the user making use of epsilon-greedy, and choosing to explore a proportion of the time. The expected reward for the proximal state is greater than the distal state. This is because  choosing actions from the distal state gives two chances of exploring before reaching the reward, while from the proximal state, there is only one chance of exploring before the reward.

Questions 

- What is the main difference between the vending machine task and the Iowa Gambling task? Why do we need Q-learning for one and not the other?

- Do you believe that you discount future rewards? Give an example?

- What other tasks might be modelled with an MDP?

- What else might bound human performance on this task?


### References:

Daw, N. D., Niv, Y. and Dayan, P. (2005), ‘Uncertainty-based competition between pre- frontal and dorsolateral striatal systems for behavioral control’, Nature neuroscience 8(12), 1704–1711.

Mic. (2016), ‘Getting AI smarter with Q-learning: a simple first step in Python’, http://firsttimeprogrammer.blogspot.co.uk/2016/09/getting-ai-smarter-with-q-learning.html. [Online; accessed 25 January 2017].

Sutton, R. S. and Barto, A. G. (1998), Reinforcement learning: An introduction, Vol. 1, MIT press Cambridge.